Importing Libraries

In [24]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

Dataset

In [25]:
fraudTrain = pd.read_csv('fraudTrain.csv')
fraudTest = pd.read_csv('fraudTest.csv')

In [26]:
fraudTrain.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495.0,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1.325376e+09,36.011293,-82.048315,0.0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149.0,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1.325376e+09,49.159047,-118.186462,0.0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154.0,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1.325376e+09,43.150704,-112.154481,0.0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939.0,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1.325376e+09,47.034331,-112.561071,0.0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99.0,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1.325376e+09,38.674999,-78.632459,0.0


In [27]:
fraudTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 635557 entries, 0 to 635556
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             635557 non-null  int64  
 1   trans_date_trans_time  635557 non-null  object 
 2   cc_num                 635557 non-null  int64  
 3   merchant               635557 non-null  object 
 4   category               635557 non-null  object 
 5   amt                    635557 non-null  float64
 6   first                  635557 non-null  object 
 7   last                   635557 non-null  object 
 8   gender                 635557 non-null  object 
 9   street                 635557 non-null  object 
 10  city                   635556 non-null  object 
 11  state                  635556 non-null  object 
 12  zip                    635556 non-null  float64
 13  lat                    635556 non-null  float64
 14  long                   635556 non-nu

Data Preprocessing

In [28]:
fraudTrain.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     1
state                    1
zip                      1
lat                      1
long                     1
city_pop                 1
job                      1
dob                      1
trans_num                1
unix_time                1
merch_lat                1
merch_long               1
is_fraud                 1
dtype: int64

In [29]:
fraudTest.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [30]:
cols_to_drop = ['Unnamed: 0','cc_num','merchant','first','last','trans_num','unix_time','street','category']
fraudTrain.drop(columns=cols_to_drop,inplace = True)
fraudTest.drop(columns=cols_to_drop,inplace = True)

In [31]:
fraudTrain.shape

(635557, 14)

In [32]:
fraudTest.shape

(555719, 14)

In [33]:
fraudTrain['lat_dist'] = abs(round(fraudTrain['merch_lat']-fraudTrain['lat'], 2))
fraudTrain['long_dist'] = abs(round(fraudTrain['merch_long']-fraudTrain['long'], 2))

fraudTest['lat_dist'] = abs(round(fraudTest['merch_lat']-fraudTest['lat'], 2))
fraudTest['long_dist'] = abs(round(fraudTest['merch_long']-fraudTest['long'], 2))

In [34]:
cols_to_drop = ['trans_date_trans_time','city','lat','long','job','dob','merch_lat','merch_long','state']
fraudTrain.drop(columns=cols_to_drop,inplace = True)
fraudTest.drop(columns=cols_to_drop,inplace = True)

In [35]:
fraudTrain.gender =[1 if value == "M" else 0 for value in fraudTrain.gender]
fraudTest.gender =[1 if value == "M" else 0 for value in fraudTest.gender]

In [36]:
X_train = fraudTrain.drop('is_fraud', axis=1)
X_test = fraudTest.drop('is_fraud', axis=1)
y_train = fraudTrain['is_fraud']
y_test = fraudTest['is_fraud']

In [37]:
X_train = X_train.dropna()
y_train = y_train[y_train.index.isin(X_train.index)]

X_test = X_test.dropna()
y_test = y_test[y_test.index.isin(X_test.index)]

In [38]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

Models

In [ ]:
#logistic regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [40]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 1.00


In [41]:
#decision tree classifier
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state = 45)
dtc.fit(X_train,y_train)
y_pred = dtc.predict(X_test)

In [42]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.99


In [43]:
#random forest classifier
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
y_pred = rfc.predict(X_test)

In [44]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 1.00
